In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'dataset/mnist-jpg/mnist-jpg/train/'
test_dir = 'dataset/mnist-jpg/mnist-jpg/test/'
img_gen = ImageDataGenerator(rescale=1.0/255)

data_gen = img_gen.flow_from_directory(
    train_dir,
    target_size=(32,32),
    batch_size=5000,
    shuffle=True,
    class_mode='categorical'
)

x_train, y_train = data_gen[0]
x_test, y_test = data_gen[2]

from tensorflow.keras.applications import VGG16
path = 'dataset/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

vgg_model = VGG16(weights=path, include_top=False, input_shape=(32,32,3))
for layer in vgg_model.layers:
    layer.trainable = False

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout
custom_classifier = keras.Sequential([
    Flatten(input_shape=(1,1,512)),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model = keras.Sequential([
    vgg_model,
    custom_classifier
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
model.fit(x_train, y_train, batch_size=100, epochs=1, validation_data=(x_test, y_test))

for layer in vgg_model.layers[:-4]:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=1000, epochs=1, validation_data=(x_test, y_test))

loss, acc = model.evaluate(x_test, y_test)
print(loss, " ", acc)

pred = model.predict(x_test)
labels = list(data_gen.class_indices.keys())

import matplotlib.pyplot as plt
import numpy as np
plt.imshow(x_test[10])
plt.title(str(labels[np.argmax(pred[10])]))
print(str(labels[np.argmax(y_test[10])]))
y_test[10]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/mnist-jpg/mnist-jpg/train/'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Lambda
import tensorflow as tf

# Load MNIST data directly from Keras (automatically downloaded if missing)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape input to fit VGG16 expected input with 3 channels and size 32x32
x_train = np.stack([x_train] * 3, axis=-1)
x_test = np.stack([x_test] * 3, axis=-1)
x_train = tf.image.resize(x_train, (32, 32)).numpy()
x_test = tf.image.resize(x_test, (32, 32)).numpy()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# One-hot encode labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load VGG16 base model without top layers and freeze layers initially
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in vgg_model.layers:
    layer.trainable = False

# Build custom classifier on top of VGG16
model = Sequential([
    vgg_model,
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)

# Evaluate model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Plot a sample image and prediction
predictions = model.predict(x_test)
plt.imshow(x_test[0])
plt.title(f"Predicted: {np.argmax(predictions[0])}, Actual: {np.argmax(y_test[0])}")
plt.axis('off')
plt.show()


MemoryError: Unable to allocate 703. MiB for an array with shape (60000, 32, 32, 3) and data type float32